<img align="center" src="images/course.png" width="800">

# 16720 (B)  3D Reconstruction - Assignment 5 - q3
    Instructor: Kris                          TAs: Zen (Lead), Yan, Rawal, Wen-Hsuan, Paritosh, Qichen

In [1]:
# Helper functions for this assignment. DO NOT MODIFY!!!
"""
Helper functions.

Written by Chen Kong, 2018.
Modified by Zhengyi (Zen) Luo, 2021
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
import cv2
import nbimporter
from q1 import eightpoint, sevenpoint, camera2, _epipoles, calc_epi_error, toHomogenous, _singularize
from q2 import find_M2, plot_3D

def plot_3D_dual(P_before, P_after):
    matplotlib.use('TkAgg')
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title("Blue: before; red: after")
    ax.scatter(P_before[:,0], P_before[:,1], P_before[:,2], c = 'blue')
    ax.scatter(P_after[:,0], P_after[:,1], P_after[:,2], c='red')
    while True:
        x, y = plt.ginput(1, mouse_stop=2)[0]
        plt.draw()



## Q3: Bundle Adjustment
Bundle Adjustment is commonly used as the last step of every feature-based 3D reconstruction algorithm. Given a set of images depicting a number of 3D points from different viewpoints, bundle adjustment is the process of simultaneously refining the 3D coordinates along with the camera parameters. It minimizes reprojection error, which is the squared sum of distances between image points and predicted points. In this section, you will implement bundle adjustment algorithm by yourself. Specifically,


- In Q3.1, you need to implement a RANSAC algorithm to estimate the fundamental matrix F and all the inliers.
- In Q3.2, you will need to write code to parameterize Rotation matrix $\mathbf{R}$ using [Rodrigues formula](https://en.wikipedia.org/wiki/Rodrigues\%27\_formul) (Please check [this pdf](https://www2.cs.duke.edu/courses/fall13/compsci527/notes/rodrigues.pdf) for a detailed explanation), which will enable the joint optimization process for Bundle Adjustment.
- Q3.3, you will need to first write down the objective function in rodriguesResidual, and do the bundleAdjustment.

### Q3.1 RANSAC for Fundamental Matrix Recovery (15 pt implementation)

In some real world applications, manually determining correspondences is infeasible and often there will be noisy correspondences. Fortunately, the RANSAC method seen (and implemented in previous assignments) in class can be applied to the problem of fundamental matrix estimation.

Implement the above algorithm with the signature:
```
[F, inliers] = ransacF(pts1, pts2, M)
```

where `M` is defined in the same way as when we calculate the fundamental matrix and inliers is a boolean vector of size equivalent to the number of points. Here inliers are set to true only for the points that satisfy the threshold defined for the given fundamental matrix F.

We have provided some noisy coorespondances in some\_corresp\_noisy.npz in which around $75\%$ of the points are inliers. Compare the result of RANSAC with the result of the eight-point algorithm when ran on the noisy correspondences. 

**Hints:** Use the seven point to compute the fundamental matrix from the minimal set of points. Then compute the inliers, and refine your estimate using all the inliers.


In [2]:
def ransacF(pts1, pts2, M):
    '''
    Q3.1: RANSAC method.
        Input:  pts1, Nx2 Matrix
                pts2, Nx2 Matrix
                M, a scaler parameter
        Output: F, the fundamental matrix
                inlier_curr, Nx1 bool vector set to true for inliers
    ***
    Hints:
    (1) You can use the calc_epi_error from q1 with threshold to calculate inliers. Tune the threshold based on 
        the results/expected number of inliners. You can also define your own metric. 
    (2) Use the seven point alogrithm to estimate the fundamental matrix as done in q1
    (3) Choose the resulting F that has the most number of inliers
    (4) You can increase the nIters to bigger/smaller values
        
    '''
    N = pts1.shape[0]
    pts1_homo, pts2_homo = toHomogenous(pts1), toHomogenous(pts2)
    threshold = 3
    max_iters = 1000
    best_inliers = None
    
    for i in range(max_iters):
        indices = np.random.randint(0, N, 7)
        curr_Farray = sevenpoint(pts1[indices], pts2[indices], M)
        
        for j in range(len(curr_Farray)):
            errors = calc_epi_error(pts1_homo, pts2_homo, curr_Farray[j])
            curr_inliers = np.argwhere(errors < threshold)
            
            if (best_inliers is None or curr_inliers.shape[0] > best_inliers.shape[0]):
                best_inliers = curr_inliers
#                 print(best_inliers.shape)
                F = curr_Farray[j]
    
    return F, best_inliers


np.random.seed(0)
np.set_printoptions(precision=4, suppress=1)
correspondence = np.load('data/some_corresp_noisy.npz') # Loading noisy correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')
F, inliners = ransacF(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))


In [3]:
# Full set of tests; you will get full points for coding if passing the following tests. 
assert(np.sum(inliners) > len(pts1) * 0.7)

### Q3.2 Rodrigues and Invsere Rodrigues (10 pt implementation)
So far we have independently solved for the camera matrix, $\mathbf{M}_j$ and 3D projections, $\textbf{w}_i$. In bundle adjustment, we will jointly optimize the reprojection error with respect to the points $\textbf{w}_i$ and the camera matrix $\textbf{w}_j$.

$$
err = \sum_{ij} ||\textbf{x}_{ij} - Proj(\mathbf{C}_j, \textbf{w}_i)||^2,
$$
where $\textbf{w}_j = \mathbf{K}_j \mathbf{M}_j$.

For this homework, we are going to only look at optimizing the extrinsic matrix. The rotation matrix forms the Lie Group $\textbf{SO}(3)$ that doesn't satisfy the addition operation so it cannot be directly optimized. Instead, we parameterize the rotation matrix to axis angle using Rodrigues formula to the Lie Algebra $\mathfrak{so}(3)$, which is defined in $\mathbb{R}^3$. through which the least squares optimization process can be done to optimize the axis angle. Try to implement function

```
R = rodrigues(r)
```

as well as the inverse function that converts a rotation matrix $\mathbf{R}$ to a Rodrigues vector $\mathbf{r}$

```
r = invRodrigues(R)
```

Please refer to [Rodrigues formula](https://en.wikipedia.org/wiki/Rodrigues\%27\_formul)  and [this pdf](https://www2.cs.duke.edu/courses/fall13/compsci527/notes/rodrigues.pdf) for reference.


In [4]:
def rodrigues(r):
    '''
    Q3.2: Rodrigues formula.
        Input:  r, a 3x1 vector
        Output: R, a 3x3 rotation matrix
    '''
    
    theta = np.sqrt(np.sum(r ** 2))
    u = r / theta
    
#     print(u[2])
    u_x = np.array([[0, - u[2], u[1]],
                    [u[2], 0, - u[0]],
                    [- u[1], u[0], 0]])
    
    u = u.reshape(-1, 1)
    
    R = np.cos(theta) * np.eye(u.shape[0]) + (1 - np.cos(theta)) * (u @ u.T) + np.sin(theta) * u_x
    
#     print(R)
    
    return R


def invRodrigues(R):
    '''
    Q5.2: Inverse Rodrigues formula.
        Input:  R, a 3x3 rotation matrix
        Output: r, a 3x1 vector
    '''
    
    A = (R - R.T) / 2
    phi = np.array([[A[2, 1], A[0, 2], A[1, 0]]]).T    
    s = np.sqrt(np.sum(phi ** 2))
    c = (R[0, 0] + R[1, 1] + R[2, 2] - 1) / 2
    v = R[:, 0] + np.eye(R.shape[0])[:, 0]
    
    u = phi / s
    theta = np.arctan2(s, c)
    
    r = theta * u
    
    return r.flatten()


In [5]:
# Simple Tests to verify your implmentation:
from scipy.spatial.transform import Rotation as sRot
rotVec = sRot.random()
mat = rodrigues(rotVec.as_rotvec())

assert(np.linalg.norm(rotVec.as_rotvec() - invRodrigues(mat)) < 1e-3)
assert(np.linalg.norm(rotVec.as_matrix() - mat) < 1e-3)



In [6]:
# Hidden Tests

### Q3.3 Bundle Adjustment (10 pt writeup)

In this section, you need to implement the bundle adjustment algorithm. Using the parameterization you implemented in the last question, write an objective function for the extrinsic optimization:

```
residuals = rodriguesResidual(K1, M1, p1, K2, p2, x)
```
where x is the flattened concatenation of $\mathbf{w}$, $\mathbf{r}_2$, and $\mathbf{t}_2$.
$\mathbf{w}$ are the 3D points; $\mathbf{r}_2$ and $\mathbf{t}_2$ are the rotation (in the Rodrigues vector form) and translation vectors associated with the projection matrix $\mathbf{M}_2$; $p1$ and $p2$ are 2D coordinates of points in image 1 and 2, respectively. The `residuals` are the difference between the original image projections and the estimated projections (the square of $2$-norm of this vector corresponds to the error we computed in Q3.2):
```
residuals = numpy.concatenate([(p1-p1').reshape([-1]), (p2-p2').reshape([-1])])
```

Use this objective function and Scipy's nonlinear least squares optimizer $\texttt{leastsq}$ write a function to optimize for the best extrinsic matrix and 3D points using the inlier correspondences from some_corresp_noisy.npz and the RANSAC estimate of the extrinsics and 3D points as an initialization.

```
[M2, w, o1, o2] = bundleAdjustment(K1, M1, p1, K2, M2_init, p2, p_init)
```

Try to extract the rotation and translation from `M2_init`, then use `invRodrigues` you implemented previously to transform the rotation, concatenate it with translation and the 3D points, then the concatenate vector are variables to be optimized. After obtaining optimized vector, decompose it back to rotation using `Rodrigues` you implemented previously, translation and 3D points coordinates.

<span style='color:red'>**Output:**</span> In your write-up: include an image of output of the `plot_3D_dual` function by passing in the original 3D points and the optimized points. Also include the before and after reprojection error for the `rodriguesResidual` function.



In [7]:
def essentialMatrix(F, K1, K2):
    '''
    Q1.1: Compute the essential matrix E given the fundamental matrix and camera intrinsics
        Input:  F, fundamental matrix
                K1, internal camera calibration matrix of camera 1
                K2, internal camera calibration matrix of camera 2
        Output: E, the essential matrix
    '''
    
    return K2.T @ F @ K1


def triangulate(C1, pts1, C2, pts2):
    '''
    Q2.2: Triangulate a set of 2D coordinates in the image to a set of 3D points.
        Input:  C1, the 3x4 camera matrix
                pts1, the Nx2 matrix with the 2D image coordinates per row
                C2, the 3x4 camera matrix
                pts2, the Nx2 matrix with the 2D image coordinates per row
        Output: P, the Nx3 matrix with the corresponding 3D points per row
                err, the reprojection error.
    
    ***
    Hints:
    (1) For every input point, form A using the corresponding points from pts1 & pts2 and C1 & C2
    (2) Solve for the least square solution using np.linalg.svd
    (3) Calculate the reprojection error using the calculated 3D points and C1 & C2 (do not forget to convert from 
        homogeneous coordinates to non-homogeneous ones)
    (4) Keep track of the 3D points and projection error, and continue to next point 
    (5) You do not need to follow the exact procedure above. 
    '''
    
    P = []
    err = 0.
    
    for i in range(pts1.shape[0]):
        A = np.array([[pts1[i, 0] * C1[2, 0] - C1[0, 0], pts1[i, 0] * C1[2, 1] - C1[0, 1], pts1[i, 0] * C1[2, 2] - C1[0, 2], pts1[i, 0] * C1[2, 3] - C1[0, 3]],
                      [pts1[i, 1] * C1[2, 0] - C1[1, 0], pts1[i, 1] * C1[2, 1] - C1[1, 1], pts1[i, 1] * C1[2, 2] - C1[1, 2], pts1[i, 1] * C1[2, 3] - C1[1, 3]],
                      [pts2[i, 0] * C2[2, 0] - C2[0, 0], pts2[i, 0] * C2[2, 1] - C2[0, 1], pts2[i, 0] * C2[2, 2] - C2[0, 2], pts2[i, 0] * C2[2, 3] - C2[0, 3]],
                      [pts2[i, 1] * C2[2, 0] - C2[1, 0], pts2[i, 1] * C2[2, 1] - C2[1, 1], pts2[i, 1] * C2[2, 2] - C2[1, 2], pts2[i, 1] * C2[2, 3] - C2[1, 3]]])

        _, _, v = np.linalg.svd(A)
        x = v[-1].reshape(-1, 1)
        x /= x[-1]
        
        pred_p1 = C1 @ x
        pred_p1 /= pred_p1[-1]
        pred_p2 = C2 @ x
        pred_p2 /= pred_p2[-1]
        
        assert pred_p1[-1] == 1
        assert pred_p2[-1] == 1
        
        # In case the 3D point is projected behind the camera center i.e.
        # z-coordinate is 0.
        if (x[2] < 0):
            return None, np.finfo('float').max
        
        err += np.sum((pts1[i] - pred_p1[: -1].flatten()) ** 2 + (pts2[i] - pred_p2[: -1].flatten()) ** 2)
        
        P.append(x[: -1])
    
    P = np.stack(P).reshape(-1, 3)
    
    return P, err
    

def find_M2(F, pts1, pts2, intrinsics):
    '''
    Q2.2: Function to find the camera2's projective matrix given correspondences
        Input:  F, the pre-computed fundamental matrix
                pts1, the Nx2 matrix with the 2D image coordinates per row
                pts2, the Nx2 matrix with the 2D image coordinates per row
                intrinsics, the intrinsics of the cameras, load from the .npz file
        Output: [M2, C2, P] the computed M2 (3x4) camera projective matrix, C2 (3x4) K2 * M2, and the 3D points P (Nx3)
    
    ***
    Hints:
    (1) Loop through the 'M2s' and use triangulate to calculate the 3D points and projection error. Keep track 
        of the projection error through best_error and retain the best one. 
    (2) Remember to take a look at camera2 to see how to correctly reterive the M2 matrix from 'M2s'. 

    '''
    K1 = intrinsics["K1"]
    K2 = intrinsics["K2"]
    E = essentialMatrix(F, K1, K2)
    M1 = np.hstack((np.identity(3), np.zeros(3)[:,np.newaxis]))
    C1 = K1.dot(M1)
    M2s = camera2(E)
    best_error = np.finfo('float').max
    P = None
    M2 = None
    
    for i in range(M2s.shape[-1]):
        curr_P, curr_error = triangulate(C1, pts1, K2 @ M2s[:, :, i], pts2)
        
        if curr_error < best_error:
            best_error = curr_error
            P = curr_P
            M2 = M2s[:, :, i]
    
    C2 = K2 @ M2
    
    print(f"Best Error {best_error}")
    return M2, C2, P
    

In [8]:
from scipy import optimize

def rodriguesResidual(x, K1, M1, p1, K2, p2):
    '''
    Q3.3: Rodrigues residual.
        Input:
                x, the flattened concatenationg of P, r2, and t2.  
                K1, the intrinsics of camera 1
                M1, the extrinsics of camera 1
                p1, the 2D coordinates of points in image 1
                K2, the intrinsics of camera 2
                p2, the 2D coordinates of points in image 2
        Output: residuals, 4N x 1 vector, the difference between original 
                and estimated projections
    '''    
    N = p1.shape[0]
        
    x_alt = x.reshape((N + 2), 3)
    
    r = x_alt[-2].reshape(-1, 1)
    t = x_alt[-1].reshape(-1, 1)
    w = x_alt[: -2]
    w_h = np.hstack((w, np.ones(w.shape[0]).reshape(-1, 1)))
    
#     print(f'{r = }')
#     print(f'{rodrigues(r) = }')
    
    M2 = np.hstack((rodrigues(r.flatten()), t))    
    C1 = K1 @ M1
    C2 = K2 @ M2
    
#     print(M2)
    
    p1_alt = (C1 @ w_h.T).T
    p2_alt = (C2 @ w_h.T).T
    
    
    p1_alt = p1_alt / p1_alt[:, -1].reshape(-1, 1)
    p2_alt = p2_alt / p2_alt[:, -1].reshape(-1, 1)
    
    p1_alt = p1_alt[:, : -1]
    p2_alt = p2_alt[:, : -1]

    residuals = np.concatenate([(p1 - p1_alt).reshape(-1), (p2 - p2_alt).reshape(-1)])
    
#     print(residuals.shape)
    
    return residuals


def bundleAdjustment(K1, M1, p1, K2, M2_init, p2, P_init):
    '''
    Q3.3 Bundle adjustment.
        Input:  K1, the intrinsics of camera 1
                M1, the extrinsics of camera 1
                p1, the 2D coordinates of points in image 1
                K2,  the intrinsics of camera 2
                M2_init, the initial extrinsics of camera 1
                p2, the 2D coordinates of points in image 2
                P_init, the initial 3D coordinates of points
        Output: M2, the optimized extrinsics of camera 1
                P2, the optimized 3D coordinates of points
                o1, the starting objective function value with the initial input
                o2, the ending objective function value after bundle adjustment
    
    ***
    Hints:
    (1) Use the scipy.optimize.minimize function to minimize the objective function, rodriguesResidual. 
        You can try different (method='..') in scipy.optimize.minimize for best results. 
    '''
    obj_start = obj_end = 0
    
    R = M2_init[:, :-1]
    r = invRodrigues(R).reshape(1, -1)
    t = M2_init[:, -1].reshape(1, -1)
    
#     print(t)
    
    x_init = np.vstack((P_init, r, t)).flatten()
    
#     print(np.vstack((P_init, r, t)).shape)
#     print(x_init.shape)
    
    obj_start = np.sum(rodriguesResidual(x_init, K1, M1, p1, K2, p2) ** 2)
    
    x, _ = optimize.leastsq(rodriguesResidual, x_init, args=(K1, M1, p1, K2, p2))
    
    x_alt = x.reshape(-1, 3)
    r = x_alt[-2].reshape(-1, 1)
    t = x_alt[-1].reshape(-1, 1)
    P = x_alt[: -2]
    P = P / P[:, -1].reshape(-1, 1)
    M2 = np.hstack((rodrigues(r.flatten()), t))  
    
    obj_end = np.sum(rodriguesResidual(x, K1, M1, p1, K2, p2) ** 2)
    
    return M2, P, obj_start, obj_end


In [9]:
# Visualization:
np.random.seed(0)
correspondence = np.load('data/some_corresp_noisy.npz') # Loading noisy correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')

M=np.max([*im1.shape, *im2.shape])
F, inliners = ransacF(pts1, pts2, M)
pts1_inliners = pts1[inliners.squeeze(), :]
pts2_inliners = pts2[inliners.squeeze(), :]

M1 = np.hstack((np.identity(3), np.zeros(3)[:,np.newaxis]))
C1 = K1.dot(M1)
M2_init,C2, P_init = find_M2(F, pts1_inliners, pts2_inliners, intrinsics)

M2, P_final, obj_start, obj_end = bundleAdjustment(K1, M1, pts1_inliners, K2, M2_init, pts2_inliners, P_init)
print(f"Before {obj_start}, After {obj_end}")

Best Error 1776.8326296114421
Before 1776.832629611452, After 8.827926861402842


In [10]:
# Visualization:
np.random.seed(0)
correspondence = np.load('data/some_corresp_noisy.npz') # Loading noisy correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')

M=np.max([*im1.shape, *im2.shape])
F, inliners = ransacF(pts1, pts2, M)
pts1_inliners = pts1[inliners.squeeze(), :]
pts2_inliners = pts2[inliners.squeeze(), :]

M1 = np.hstack((np.identity(3), np.zeros(3)[:,np.newaxis]))
C1 = K1.dot(M1)
M2_init,C2, P_init = find_M2(F, pts1_inliners, pts2_inliners, intrinsics)

M2, P_final, obj_start, obj_end = bundleAdjustment(K1, M1, pts1_inliners, K2, M2_init, pts2_inliners, P_init)
print(f"Before {obj_start}, After {obj_end}")
plot_3D_dual(P_init, P_final)


Best Error 1776.8326296114421
Before 1776.832629611452, After 8.827926861402842


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.8/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/usr/lib/python3.8/tkinter/__init__.py", line 814, in callit
    func(*args)
  File "/home/punit13/personal/venv/lib/python3.8/site-packages/matplotlib/backends/_backend_tk.py", line 476, in delayed_destroy
    self.window.destroy()
  File "/usr/lib/python3.8/tkinter/__init__.py", line 2311, in destroy
    for c in list(self.children.values()): c.destroy()
  File "/usr/lib/python3.8/tkinter/__init__.py", line 2583, in destroy
    Misc.destroy(self)
  File "/usr/lib/python3.8/tkinter/__init__.py", line 640, in destroy
    self.tk.deletecommand(name)
_tkinter.TclError: can't delete Tcl command


TclError: 